# Data Views for Agent Workflows

This notebook sets up the complete data foundation for the dual-agent healthcare claims system.

**Prerequisites:**
- Access to `CLAIMS_DEMO` database and `CLAIMS_HOSPITAL_CLAIMS__REMITS_DATA` marketplace
- Cortex services enabled
- PCS_CODES table already exists in CLAIMS_DEMO.PUBLIC

**What this builds:**
- Patient data tables for Builder Agent input gathering
- Procedure code references with marketplace intelligence
- Denial patterns and Insurance Agent toolkit
- Success patterns for Builder Agent optimization
- Cigna policy rules for both agents

**IMPORTANT:** Run cells in order - some views depend on tables/views created in earlier cells.


## Step 1: Patient Data Setup

Create patient data table and load sample Cigna members for demo scenarios.


In [ ]:
CREATE TABLE IF NOT EXISTS CLAIMS_DEMO.PUBLIC.PATIENTS (
    PATIENT_ID VARCHAR(50) PRIMARY KEY,
    FIRST_NAME VARCHAR(100),
    LAST_NAME VARCHAR(100),
    DATE_OF_BIRTH DATE,
    GENDER VARCHAR(10),
    INSURANCE_PROVIDER VARCHAR(100),
    POLICY_NUMBER VARCHAR(50),
    GROUP_NUMBER VARCHAR(50),
    PRIMARY_CARE_PHYSICIAN VARCHAR(200),
    MEDICAL_HISTORY_SUMMARY TEXT,
    ALLERGIES TEXT,
    CURRENT_MEDICATIONS TEXT,
    CREATED_DATE TIMESTAMP DEFAULT CURRENT_TIMESTAMP()
);


In [ ]:
INSERT INTO CLAIMS_DEMO.PUBLIC.PATIENTS (PATIENT_ID, FIRST_NAME, LAST_NAME, DATE_OF_BIRTH, GENDER, INSURANCE_PROVIDER, POLICY_NUMBER, GROUP_NUMBER, PRIMARY_CARE_PHYSICIAN, MEDICAL_HISTORY_SUMMARY, ALLERGIES, CURRENT_MEDICATIONS) VALUES 
('PAT_001', 'John', 'Smith', '1975-03-15', 'Male', 'Cigna', 'CGN123456789', 'CIGNA_GRP001', 'Dr. Sarah Johnson', 'Hypertension, Type 2 Diabetes', 'Penicillin', 'Metformin 500mg, Lisinopril 10mg'),
('PAT_002', 'Maria', 'Garcia', '1982-07-22', 'Female', 'Cigna', 'CGN234567890', 'CIGNA_STANDARD', 'Dr. Michael Chen', 'Osteoarthritis, Previous knee surgery 2019', 'Sulfa drugs', 'Ibuprofen 400mg PRN'),
('PAT_003', 'Robert', 'Wilson', '1968-11-08', 'Male', 'Cigna', 'CGN345678901', 'CIGNA_CORP500', 'Dr. Lisa Park', 'Chronic back pain, Sleep apnea', 'None known', 'Gabapentin 300mg, CPAP therapy'),
('PAT_004', 'Jennifer', 'Davis', '1990-05-12', 'Female', 'Cigna', 'CGN456789012', 'CIGNA_SMALL250', 'Dr. David Kim', 'Asthma, Seasonal allergies', 'Shellfish', 'Albuterol inhaler, Claritin 10mg'),
('PAT_005', 'Thomas', 'Anderson', '1955-09-30', 'Male', 'Cigna', 'CGN567890123', 'CIGNA_STANDARD', 'Dr. Rachel Adams', 'Coronary artery disease, High cholesterol', 'Codeine', 'Atorvastatin 40mg, Metoprolol 50mg');


## Step 2: Procedure Code Intelligence

Create procedure reference views combining marketplace data with detailed descriptions.


In [ ]:
CREATE OR REPLACE VIEW CLAIMS_DEMO.PUBLIC.PROCEDURE_CODES_REFERENCE AS
SELECT DISTINCT
    CPTCODE,
    COUNT(*) as USAGE_COUNT,
    MIN(CLAIMID) as EXAMPLE_CLAIM_ID
FROM CLAIMS_HOSPITAL_CLAIMS__REMITS_DATA.ISTG.CPTDETAIL 
WHERE CPTCODE IS NOT NULL 
GROUP BY CPTCODE
ORDER BY USAGE_COUNT DESC;


In [ ]:
CREATE OR REPLACE VIEW CLAIMS_DEMO.PUBLIC.DIAGNOSIS_CODES_REFERENCE AS
SELECT DISTINCT
    DIAGCODE,
    COUNT(*) as USAGE_COUNT,
    MIN(CLAIMID) as EXAMPLE_CLAIM_ID
FROM CLAIMS_HOSPITAL_CLAIMS__REMITS_DATA.ISTG.DIAGNOSISDETAIL 
WHERE DIAGCODE IS NOT NULL 
GROUP BY DIAGCODE
ORDER BY USAGE_COUNT DESC;


In [ ]:
CREATE TABLE IF NOT EXISTS CLAIMS_DEMO.PUBLIC.COMMON_PROCEDURES (
    PROCEDURE_CODE VARCHAR(20),
    PROCEDURE_NAME VARCHAR(500),
    MEDICAL_SPECIALTY VARCHAR(100),
    TYPICAL_COST_RANGE VARCHAR(50),
    COMMON_DIAGNOSES TEXT,
    CIGNA_COVERAGE_NOTES TEXT,
    CREATED_DATE TIMESTAMP DEFAULT CURRENT_TIMESTAMP()
);


In [ ]:
INSERT INTO CLAIMS_DEMO.PUBLIC.COMMON_PROCEDURES (PROCEDURE_CODE, PROCEDURE_NAME, MEDICAL_SPECIALTY, TYPICAL_COST_RANGE, COMMON_DIAGNOSES, CIGNA_COVERAGE_NOTES) VALUES
('85025', 'Complete Blood Count (CBC)', 'Laboratory', '$25-75', 'Anemia, Infection, General Health Screening', 'Covered 100% for preventive care, may require copay for diagnostic'),
('80053', 'Comprehensive Metabolic Panel (CMP)', 'Laboratory', '$50-150', 'Diabetes, Kidney Disease, Electrolyte Imbalance', 'Covered annually for preventive, additional tests may require prior auth'),
('36415', 'Blood Draw/Venipuncture', 'Laboratory', '$15-40', 'Required for any blood test', 'Standard coverage with lab services'),
('99283', 'Emergency Department Visit - Moderate Complexity', 'Emergency Medicine', '$400-800', 'Chest Pain, Shortness of Breath, Moderate Injuries', 'Covered after deductible, no prior auth required for emergencies'),
('G0463', 'Hospital Outpatient Clinic Visit', 'General Medicine', '$150-300', 'Follow-up Care, Chronic Disease Management', 'Standard outpatient coverage applies'),
('96372', 'Injection - Subcutaneous/Intramuscular', 'Various', '$25-100', 'Vaccinations, Medication Administration', 'Covered per pharmacy benefits for medications'),
('99214', 'Office Visit - Established Patient, Detailed', 'Primary Care', '$200-350', 'Chronic Disease Management, Complex Conditions', 'Standard office visit coverage'),
('93005', 'Electrocardiogram (EKG)', 'Cardiology', '$100-250', 'Chest Pain, Heart Palpitations, Cardiac Screening', 'Covered for diagnostic purposes, may need prior auth for screening');


## Step 3: Insurance Agent Toolkit - Denial Analysis

Create denial patterns and rebuttal resources for the Insurance Agent.


In [ ]:
CREATE OR REPLACE VIEW CLAIMS_DEMO.PUBLIC.DENIAL_PATTERNS AS
SELECT 
    EOBPAYERNAME as PAYER_NAME,
    DENIALCATEGORYID,
    COUNT(*) as DENIAL_COUNT,
    AVG(BILLEDAMOUNT) as AVG_BILLED_AMOUNT,
    AVG(DENIEDADJUSTMENT) as AVG_DENIED_AMOUNT,
    AVG(DENIEDADJUSTMENT/NULLIF(BILLEDAMOUNT,0)) * 100 as AVG_DENIAL_PERCENTAGE,
    MIN(DENIEDADJUSTMENT) as MIN_DENIED,
    MAX(DENIEDADJUSTMENT) as MAX_DENIED
FROM CLAIMS_HOSPITAL_CLAIMS__REMITS_DATA.ISTG.EOBDETAIL 
WHERE DENIEDADJUSTMENT > 0 
    AND BILLEDAMOUNT > 0
GROUP BY EOBPAYERNAME, DENIALCATEGORYID
ORDER BY DENIAL_COUNT DESC;


In [ ]:
CREATE OR REPLACE VIEW CLAIMS_DEMO.PUBLIC.CIGNA_DENIAL_BEHAVIOR AS
SELECT 
    CASE 
        WHEN UPPER(EOBPAYERNAME) LIKE '%CIGNA%' THEN 'CIGNA_ACTUAL'
        WHEN UPPER(EOBPAYERNAME) LIKE '%BLUE%' THEN 'BLUE_CROSS_PATTERN'
        WHEN UPPER(EOBPAYERNAME) LIKE '%MEDICARE%' THEN 'MEDICARE_PATTERN'
        WHEN UPPER(EOBPAYERNAME) LIKE '%AETNA%' THEN 'AETNA_PATTERN'
        ELSE 'OTHER_PAYER'
    END as PAYER_TYPE,
    DENIALCATEGORYID,
    COUNT(*) as DENIAL_COUNT,
    ROUND(AVG(BILLEDAMOUNT), 2) as AVG_BILLED,
    ROUND(AVG(DENIEDADJUSTMENT), 2) as AVG_DENIED,
    ROUND(AVG(DENIEDADJUSTMENT/NULLIF(BILLEDAMOUNT,0)) * 100, 1) as DENIAL_RATE_PCT
FROM CLAIMS_HOSPITAL_CLAIMS__REMITS_DATA.ISTG.EOBDETAIL 
WHERE DENIEDADJUSTMENT > 0 AND BILLEDAMOUNT > 0
GROUP BY 1, 2
HAVING COUNT(*) >= 1
ORDER BY DENIAL_COUNT DESC;


In [ ]:
CREATE OR REPLACE VIEW CLAIMS_DEMO.PUBLIC.INSURANCE_AGENT_TOOLKIT AS
SELECT 
    dr.DENIAL_REASON_ID,
    dr.DENIAL_CATEGORY,
    dr.DENIAL_DESCRIPTION,
    dr.REBUTTAL_TEMPLATE,
    dp.DENIAL_COUNT as MARKETPLACE_PRECEDENT,
    dp.AVG_DENIAL_PERCENTAGE as TYPICAL_DENIAL_RATE,
    CASE 
        WHEN dp.DENIAL_COUNT > 10 THEN 'COMMON_REASON'
        WHEN dp.DENIAL_COUNT > 3 THEN 'MODERATE_REASON'
        WHEN dp.DENIAL_COUNT > 0 THEN 'RARE_REASON'
        ELSE 'NO_MARKETPLACE_DATA'
    END as PRECEDENT_STRENGTH
FROM CLAIMS_DEMO.PUBLIC.DENIAL_REASONS dr
LEFT JOIN CLAIMS_DEMO.PUBLIC.DENIAL_PATTERNS dp ON dr.DENIAL_CATEGORY = dp.PAYER_NAME
ORDER BY dr.DENIAL_REASON_ID;


In [ ]:
CREATE TABLE IF NOT EXISTS CLAIMS_DEMO.PUBLIC.DENIAL_REASONS (
    DENIAL_REASON_ID VARCHAR(10) PRIMARY KEY,
    DENIAL_CATEGORY VARCHAR(100),
    DENIAL_DESCRIPTION TEXT,
    TYPICAL_CLAIM_ISSUES TEXT,
    CIGNA_SPECIFIC_POLICY TEXT,
    REBUTTAL_TEMPLATE TEXT,
    CREATED_DATE TIMESTAMP DEFAULT CURRENT_TIMESTAMP()
);


In [ ]:
INSERT INTO CLAIMS_DEMO.PUBLIC.DENIAL_REASONS (DENIAL_REASON_ID, DENIAL_CATEGORY, DENIAL_DESCRIPTION, TYPICAL_CLAIM_ISSUES, CIGNA_SPECIFIC_POLICY, REBUTTAL_TEMPLATE) VALUES
('D001', 'Prior Authorization Required', 'Service requires prior authorization from Cigna before treatment', 'Missing prior auth number, procedure not pre-approved', 'Cigna requires prior authorization for procedures over $1000 or specialty services', 'CLAIM DENIED: Prior authorization required per Cigna policy section 4.2.1. Please obtain authorization and resubmit with authorization number.'),
('D002', 'Medical Necessity Not Established', 'Clinical documentation does not support medical necessity', 'Insufficient physician notes, diagnosis does not justify procedure', 'Cigna requires clear medical necessity documentation per clinical guidelines', 'CLAIM DENIED: Medical necessity not established. Please provide additional physician documentation supporting the medical necessity of this procedure.'),
('D003', 'Non-Covered Service', 'Service not covered under patient plan benefits', 'Experimental procedures, cosmetic services, excluded benefits', 'Service excluded per Cigna plan documents and member certificate', 'CLAIM DENIED: Service not covered under member plan. Refer to plan documents section 6.3 for covered benefits.'),
('D004', 'Duplicate Claim', 'Claim previously processed and paid', 'Same procedure, same date, already adjudicated', 'Cigna systems show previous payment for identical service', 'CLAIM DENIED: Duplicate claim. Previous payment issued on [DATE] for identical service. Check Cigna portal for payment details.'),
('D005', 'Incorrect Coding', 'Procedure codes do not match services rendered', 'Wrong CPT code, unbundling, modifier issues', 'Cigna follows CMS coding guidelines and local coverage determinations', 'CLAIM DENIED: Incorrect procedure coding. CPT code [CODE] does not match documented service. Please review and resubmit with correct codes.'),
('D006', 'Frequency Limitation Exceeded', 'Service exceeds allowed frequency per plan year', 'Too many visits, tests repeated too frequently', 'Cigna allows specific frequencies per clinical guidelines', 'CLAIM DENIED: Frequency limitation exceeded. Member has already received [X] services this plan year. Benefit limit: [Y] per year.');


## Step 4: Success Patterns for Builder Agent

Create views of successful claims for the Builder Agent to learn optimal patterns.


In [ ]:
CREATE OR REPLACE VIEW CLAIMS_DEMO.PUBLIC.BUILDER_AGENT_SUCCESS_GUIDE AS
SELECT 
    CHARGEPROCEDURECODE,
    COUNT(*) as SUCCESS_COUNT,
    ROUND(AVG(PAYMENT_RATE_PCT), 1) as AVG_PAYMENT_RATE,
    ROUND(AVG(BILLEDAMOUNT), 2) as AVG_BILLED_AMOUNT,
    ROUND(AVG(PAIDAMOUNT), 2) as AVG_PAID_AMOUNT,
    LISTAGG(DISTINCT EOBPAYERNAME, ', ') as SUCCESSFUL_PAYERS,
    LISTAGG(DISTINCT PLACEOFSERVICECODE, ', ') as COMMON_SERVICE_LOCATIONS,
    CASE 
        WHEN COUNT(*) >= 5 THEN 'HIGH_SUCCESS_PROCEDURE'
        WHEN COUNT(*) >= 2 THEN 'MODERATE_SUCCESS_PROCEDURE'
        ELSE 'LIMITED_SUCCESS_DATA'
    END as SUCCESS_CONFIDENCE
FROM CLAIMS_DEMO.PUBLIC.SUCCESSFUL_CLAIMS_PATTERNS
GROUP BY CHARGEPROCEDURECODE
ORDER BY SUCCESS_COUNT DESC, AVG_PAYMENT_RATE DESC;


In [ ]:
CREATE OR REPLACE VIEW CLAIMS_DEMO.PUBLIC.PROCEDURE_CODES_WITH_DESCRIPTIONS AS
SELECT 
    pc.PROCEDURECODE,
    pc.FULL_DESCRIPTION,
    COALESCE(pr.USAGE_COUNT, 0) as MARKETPLACE_USAGE_COUNT,
    pr.EXAMPLE_CLAIM_ID,
    CASE 
        WHEN pr.USAGE_COUNT > 20 THEN 'HIGH_USAGE'
        WHEN pr.USAGE_COUNT > 5 THEN 'MEDIUM_USAGE' 
        WHEN pr.USAGE_COUNT > 0 THEN 'LOW_USAGE'
        ELSE 'NO_MARKETPLACE_DATA'
    END as USAGE_CATEGORY
FROM CLAIMS_DEMO.PUBLIC.PCS_CODES pc
LEFT JOIN CLAIMS_DEMO.PUBLIC.PROCEDURE_CODES_REFERENCE pr 
    ON pc.PROCEDURECODE = pr.CPTCODE
ORDER BY COALESCE(pr.USAGE_COUNT, 0) DESC, pc.PROCEDURECODE;


In [ ]:
-- Create enhanced procedure codes view combining PCS_CODES with marketplace usage
CREATE OR REPLACE VIEW CLAIMS_DEMO.PUBLIC.PROCEDURE_CODES_WITH_DESCRIPTIONS AS
SELECT 
    pc.PROCEDURECODE,
    pc.FULL_DESCRIPTION,
    COALESCE(pr.USAGE_COUNT, 0) as MARKETPLACE_USAGE_COUNT,
    pr.EXAMPLE_CLAIM_ID,
    CASE 
        WHEN pr.USAGE_COUNT > 20 THEN 'HIGH_USAGE'
        WHEN pr.USAGE_COUNT > 5 THEN 'MEDIUM_USAGE' 
        WHEN pr.USAGE_COUNT > 0 THEN 'LOW_USAGE'
        ELSE 'NO_MARKETPLACE_DATA'
    END as USAGE_CATEGORY
FROM CLAIMS_DEMO.PUBLIC.PCS_CODES pc
LEFT JOIN CLAIMS_DEMO.PUBLIC.PROCEDURE_CODES_REFERENCE pr 
    ON pc.PROCEDURECODE = pr.CPTCODE
ORDER BY COALESCE(pr.USAGE_COUNT, 0) DESC, pc.PROCEDURECODE;


In [ ]:
CREATE TABLE IF NOT EXISTS CLAIMS_DEMO.PUBLIC.CIGNA_POLICY_RULES (
    POLICY_RULE_ID VARCHAR(20) PRIMARY KEY,
    POLICY_SECTION VARCHAR(100),
    RULE_CATEGORY VARCHAR(100),
    PROCEDURE_CODES TEXT,
    COVERAGE_CRITERIA TEXT,
    PRIOR_AUTH_REQUIRED BOOLEAN,
    FREQUENCY_LIMITS VARCHAR(200),
    EXCLUSIONS TEXT,
    MEDICAL_NECESSITY_REQUIREMENTS TEXT,
    DOCUMENTATION_REQUIREMENTS TEXT,
    CREATED_DATE TIMESTAMP DEFAULT CURRENT_TIMESTAMP()
);


In [ ]:
CREATE OR REPLACE VIEW CLAIMS_DEMO.PUBLIC.CLAIM_STRENGTH_CALCULATOR AS
SELECT 
    'STRENGTH_METRICS' as METRIC_TYPE,
    COUNT(CASE WHEN DENIEDADJUSTMENT = 0 THEN 1 END) as APPROVED_CLAIMS,
    COUNT(CASE WHEN DENIEDADJUSTMENT > 0 THEN 1 END) as DENIED_CLAIMS,
    ROUND(COUNT(CASE WHEN DENIEDADJUSTMENT = 0 THEN 1 END) * 100.0 / COUNT(*), 1) as APPROVAL_RATE_PCT,
    ROUND(AVG(CASE WHEN DENIEDADJUSTMENT > 0 THEN DENIEDADJUSTMENT/NULLIF(BILLEDAMOUNT,0) END) * 100, 1) as AVG_DENIAL_PCT,
    ROUND(MEDIAN(BILLEDAMOUNT), 2) as MEDIAN_CLAIM_AMOUNT,
    ROUND(MEDIAN(CASE WHEN DENIEDADJUSTMENT > 0 THEN DENIEDADJUSTMENT END), 2) as MEDIAN_DENIAL_AMOUNT
FROM CLAIMS_HOSPITAL_CLAIMS__REMITS_DATA.ISTG.EOBDETAIL
WHERE BILLEDAMOUNT > 0;


In [ ]:
-- Verify marketplace data access (should show row counts)
SELECT 'MARKETPLACE_CLAIMS' as SOURCE, COUNT(*) as ROW_COUNT 
FROM CLAIMS_HOSPITAL_CLAIMS__REMITS_DATA.ISTG.CLAIMCHARGEDETAIL
UNION ALL
SELECT 'MARKETPLACE_DENIALS', COUNT(*) 
FROM CLAIMS_HOSPITAL_CLAIMS__REMITS_DATA.ISTG.EOBDETAIL WHERE DENIEDADJUSTMENT > 0
UNION ALL  
SELECT 'MARKETPLACE_PROCEDURES', COUNT(*) 
FROM CLAIMS_HOSPITAL_CLAIMS__REMITS_DATA.ISTG.CPTDETAIL
UNION ALL
SELECT 'PCS_CODES_AVAILABLE', COUNT(*) 
FROM CLAIMS_DEMO.PUBLIC.PCS_CODES;


In [ ]:
-- FINAL VERIFICATION: All components created successfully
-- Each count should be 1 for successful setup
SELECT 
    COUNT(CASE WHEN table_name = 'PATIENTS' THEN 1 END) as HAS_PATIENTS,
    COUNT(CASE WHEN table_name = 'COMMON_PROCEDURES' THEN 1 END) as HAS_PROCEDURES,
    COUNT(CASE WHEN table_name = 'DENIAL_REASONS' THEN 1 END) as HAS_DENIALS,
    COUNT(CASE WHEN table_name = 'CIGNA_POLICY_RULES' THEN 1 END) as HAS_POLICIES
FROM CLAIMS_DEMO.INFORMATION_SCHEMA.TABLES 
WHERE table_schema = 'PUBLIC';

-- Verify key views exist
SELECT table_name as VIEW_NAME
FROM CLAIMS_DEMO.INFORMATION_SCHEMA.VIEWS 
WHERE table_schema = 'PUBLIC' 
    AND table_name IN ('DENIAL_PATTERNS', 'SUCCESSFUL_CLAIMS_PATTERNS', 'AGENT_POLICY_LOOKUP', 'CLAIM_STRENGTH_CALCULATOR')
ORDER BY table_name;


In [ ]:
INSERT INTO CLAIMS_DEMO.PUBLIC.CIGNA_POLICY_RULES (POLICY_RULE_ID, POLICY_SECTION, RULE_CATEGORY, PROCEDURE_CODES, COVERAGE_CRITERIA, PRIOR_AUTH_REQUIRED, FREQUENCY_LIMITS, EXCLUSIONS, MEDICAL_NECESSITY_REQUIREMENTS, DOCUMENTATION_REQUIREMENTS) VALUES
('P001', 'Section 4.1 - Laboratory Services', 'Diagnostic Laboratory', '85025, 80053, 36415', 'Covered for diagnostic and preventive purposes', false, 'CBC: 2 per year preventive, additional as medically necessary. CMP: Annual preventive, quarterly for diabetes monitoring', 'Experimental genetic testing', 'Must be ordered by physician for specific medical indication', 'Physician order with diagnosis code'),
('P002', 'Section 4.2 - Emergency Services', 'Emergency Care', '99281, 99282, 99283, 99284, 99285', 'Covered 24/7 without prior authorization for true emergencies', false, 'No frequency limits for emergency care', 'Non-emergency use of emergency department may require member cost-sharing', 'Must meet prudent layperson standard for emergency', 'Emergency department documentation, triage notes'),
('P003', 'Section 5.1 - Office Visits', 'Primary Care', '99211, 99212, 99213, 99214, 99215', 'Covered for established and new patients', false, 'No specific limits, but excessive visits may require review', 'Routine physical exams more than annually (unless high-risk)', 'Must document medical decision making and patient encounter', 'SOAP notes, time documentation for time-based billing'),
('P004', 'Section 6.1 - Cardiology Services', 'Specialty Care', '93005, 93010, 93015', 'Covered for diagnostic and monitoring purposes', true, 'EKG: As medically necessary, Stress tests: Annual or as indicated', 'Screening in asymptomatic low-risk patients under age 40', 'Must have cardiac symptoms, risk factors, or follow-up indication', 'Cardiology consultation notes, clinical indication documentation'),
('P005', 'Section 7.2 - Injections and Infusions', 'Therapeutic Services', '96372, 96413, J0897', 'Covered when medically necessary and FDA-approved', false, 'Based on medication prescribing guidelines', 'Experimental drugs, cosmetic injections', 'Must be FDA-approved indication with appropriate diagnosis', 'Physician order, medication documentation, administration notes');


## Step 5: Final Views and Verification

Create final agent views and verify the complete data foundation.


In [ ]:
CREATE OR REPLACE VIEW CLAIMS_DEMO.PUBLIC.AGENT_POLICY_LOOKUP AS
SELECT 
    cpr.POLICY_RULE_ID,
    cpr.POLICY_SECTION,
    cpr.RULE_CATEGORY,
    cpr.PROCEDURE_CODES,
    cpr.COVERAGE_CRITERIA,
    cpr.PRIOR_AUTH_REQUIRED,
    cpr.FREQUENCY_LIMITS,
    cpr.MEDICAL_NECESSITY_REQUIREMENTS,
    cpr.DOCUMENTATION_REQUIREMENTS,
    COALESCE(cp.CIGNA_COVERAGE_NOTES, 'Standard coverage applies') as ADDITIONAL_NOTES
FROM CLAIMS_DEMO.PUBLIC.CIGNA_POLICY_RULES cpr
LEFT JOIN CLAIMS_DEMO.PUBLIC.COMMON_PROCEDURES cp 
    ON REGEXP_LIKE(cpr.PROCEDURE_CODES, cp.PROCEDURE_CODE)
ORDER BY cpr.POLICY_RULE_ID;


In [ ]:
-- Summary verification
SELECT 'PATIENTS' as TABLE_TYPE, COUNT(*) as ROW_COUNT FROM CLAIMS_DEMO.PUBLIC.PATIENTS
UNION ALL
SELECT 'COMMON_PROCEDURES', COUNT(*) FROM CLAIMS_DEMO.PUBLIC.COMMON_PROCEDURES  
UNION ALL
SELECT 'DENIAL_REASONS', COUNT(*) FROM CLAIMS_DEMO.PUBLIC.DENIAL_REASONS
UNION ALL
SELECT 'CIGNA_POLICY_RULES', COUNT(*) FROM CLAIMS_DEMO.PUBLIC.CIGNA_POLICY_RULES;
